# Challenge2

In [8]:
from langchain_community.chat_models import ChatOpenAI
from langchain.prompts import FewShotChatMessagePromptTemplate, ChatPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.cache import InMemoryCache, SQLiteCache
from langchain.globals import set_llm_cache, set_debug

In [9]:
#모델정의
llm = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()]
)

#cache
set_llm_cache(SQLiteCache("cache.db"))

answer_example = [
    {
        'movie': "Home Alone",
        'answer': '''
            pd: Christopher Joseph Columbus,
            main_cast: Daniel Stern, kevin, harry, pooler,
            budget: $40,000,000,
            genre: comic
        '''
    },
    {
        'movie': "Lion King",
        'answer': '''
            pd: spilberg,
            main_cast: iron, picaso, trump, haily,
            budget: $24,000,000,
            genre: action
        '''
    },
    {
        'movie': 'ponyo',
        'answer': '''
            pd: skandi copo kamoo,
            main_cast: gpoiler, bygat moro, uganda miko,
            budget: $280,000,000,
            genre: action
        '''
    },
]

#질문에 답하는 예시 정의
example_prompt = ChatPromptTemplate.from_messages([
    ('human', 'what do you know about {movie}?'),
    ('ai', '{answer}')
])

#질문에 답은 어떻게 하는건지 예시
example_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=answer_example,
)

#final_prompt
final_prompt = ChatPromptTemplate.from_messages([
    #system
    ('system', "You are a movie expert. give me short answer"),
    #example
    example_prompt,
    #question
    ('human', 'what do you know about {movie}?')
])

chain = final_prompt | llm
chain.invoke({"movie": "avatar"})


            pd: James Cameron,
            main_cast: Sam Worthington, Zoe Saldana, Sigourney Weaver,
            budget: $237,000,000,
            genre: science fiction, action, adventure

AIMessage(content='\n            pd: James Cameron,\n            main_cast: Sam Worthington, Zoe Saldana, Sigourney Weaver,\n            budget: $237,000,000,\n            genre: science fiction, action, adventure', response_metadata={'finish_reason': 'stop'}, id='run-ad93724f-36df-47fd-8fe1-9ad501ae1ad0-0')